<a href="https://colab.research.google.com/github/jaqueantunes/dados_saude/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#app

In [6]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# Configuração da página
st.set_page_config(page_title="Dashboard Obesidade", layout="wide")

st.title("📊 Análise de Dados de Obesidade")
st.markdown("Dados extraídos do repositório: [jaqueantunes/dados_saude](https://github.com/jaqueantunes/dados_saude)")

# URL direta do arquivo RAW no GitHub
url = "https://raw.githubusercontent.com/jaqueantunes/dados_saude/main/obesity.csv"

@st.cache_data
def carregar_dados():
    # Lê o CSV direto da internet
    return pd.read_csv(url)

try:
    with st.spinner('Baixando dados do GitHub...'):
        df = carregar_dados()

    st.success("✅ Dados carregados com sucesso via GitHub!")

    # Mostra os dados
    st.subheader("Visualização da Base de Dados")
    st.dataframe(df.head())

    # --- ÁREA DOS GRÁFICOS ---
    st.divider()
    st.subheader("Exploração Gráfica")

    # Seletor de colunas numéricas
    colunas_numericas = df.select_dtypes(include=['float64', 'int64']).columns

    if len(colunas_numericas) > 0:
        col_x = st.selectbox("Escolha o eixo X:", colunas_numericas, index=0)

        # Cria um gráfico simples
        fig = px.histogram(df, x=col_x, title=f"Distribuição de {col_x}")
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("Não encontrei colunas numéricas para criar gráficos automáticos.")

except Exception as e:
    st.error("❌ Erro ao carregar os dados!")
    st.write(f"Detalhes do erro: {e}")
    st.info("Dica: Verifique se o nome do arquivo no GitHub é exatamente 'obesity.csv' (letras maiúsculas/minúsculas importam).")

Overwriting app.py
